Repairing the dataset at client level.

Note: This has not been implemented in this notebook. To implement please make changes to the architecture and distribute the dataframe amongst clients before converting it into batched datasets.



In [2]:
import numpy as np
import random
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Dropout
from tensorflow.keras import optimizers
from sklearn.metrics import precision_score,recall_score, accuracy_score,confusion_matrix,f1_score
from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import History

import pandas as pd



C:\Users\riash\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\riash\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\riash\anaconda3\lib\site-packages\numpy\.libs\libopenblas.xwydx2ikjw2nmtwsfyngfuwkqu3lytcz.gfortran-win_amd64.dll
C:\Users\riash\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [20]:
### Train/Test Data seperation

file_out = pd.read_csv('Data/adult_processed.csv')
cols = []

for i in list(file_out.columns):
    if  i != 'income':
        cols.append(i)

feature_set1 = pd.read_csv('Data/train.csv')
feature_set2 = pd.read_csv('Data/test.csv')

x = feature_set1[cols].copy().values
y = feature_set1[['income']].copy().values
        
sc = StandardScaler()
X_train = sc.fit_transform(x)
y_train = y

x2 = feature_set2[cols].copy().values
y2 = feature_set2[['income']].copy().values
        
X_test = sc.fit_transform(x2)
y_test = y2


# X_test.shape, y_test.shape 
# if cols == cols_train:
#     print("yes")
# diff = np.setdiff1d(cols_train,cols)
# diff

In [21]:
# gender split

dict_users = {i: np.array([]) for i in range(10)}
data_out = []

def create_hetero_clients( image_list, label_list, start_client = 0, num_clients=10, initial='clients'):
    
    selected_inds = []

    max_y = np.argmax(label_list, axis=-1)
    sorted_zip = sorted(zip(max_y, label_list, image_list), key=lambda x: x[0])
    data = [(x,y) for _,y,x in sorted_zip]
    
    data_out = data
    
    num_shards, num_imgs = int(len(image_list)/30), 30
    idx_shard = [i for i in range(num_shards)]

    min_shard = 1
    max_shard = 60  #953/15 = 63.53
    
    random_shard_size = np.random.randint(min_shard, max_shard+1,
                                          size=(num_clients-start_client))
    random_shard_size = np.around(random_shard_size /
                                  sum(random_shard_size) * num_shards)
    random_shard_size = random_shard_size.astype(int)


    if sum(random_shard_size) > num_shards:
        
        for i in range(start_client, num_clients):
            # First assign each client 1 shard to ensure every client has
            # atleast one shard of data
            rand_set = set(np.random.choice(idx_shard, 1, replace=False))
            idx_shard = list(set(idx_shard) - rand_set)
            for rand in rand_set:
                ind1 = rand*num_imgs
                ind2 = (rand+1)*num_imgs
                if(len(dict_users[i]) == 0):
                    dict_users[i] = data[ind1: ind2]
                else:
                    dict_users[i] = np.concatenate(
                    (dict_users[i],data[ind1: ind2]),
                    axis=0)
                selected_inds.extend([[ind1, ind2]])

        random_shard_size = random_shard_size-1

        # Next, randomly assign the remaining shards
        for i in range(start_client, num_clients):
            if len(idx_shard) == 0:
                continue
#             print(random_shard_size)
            shard_size = random_shard_size[i-start_client]
            if shard_size > len(idx_shard):
                shard_size = len(idx_shard)
            rand_set = set(np.random.choice(idx_shard, shard_size,
                                            replace=False))
            idx_shard = list(set(idx_shard) - rand_set)
            for rand in rand_set:
                ind1 = rand*num_imgs
                ind2 = (rand+1)*num_imgs
                if(len(dict_users[i]) == 0):
                    dict_users[i] = data[ind1: ind2]
                else:
                    dict_users[i] = np.concatenate(
                    (dict_users[i],data[ind1: ind2]),
                    axis=0)
                selected_inds.extend([[ind1, ind2]])

    else:

        for i in range(start_client, num_clients):
#             print(random_shard_size)
            shard_size = random_shard_size[i-start_client]
#             shard_size = random_shard_size[int(i/len(random_shard_size)) - 1]
            rand_set = set(np.random.choice(idx_shard, shard_size,
                                            replace=False))
            idx_shard = list(set(idx_shard) - rand_set)
            for rand in rand_set:
                ind1 = rand*num_imgs
                ind2 = (rand+1)*num_imgs
                if(len(dict_users[i]) == 0):
                    dict_users[i] = data[ind1: ind2]
                else:
                    dict_users[i] = np.concatenate(
                    (dict_users[i],data[ind1: ind2]),
                    axis=0)
                selected_inds.extend([[ind1, ind2]])

        if len(idx_shard) > 0:
            # Add the leftover shards to the client with minimum images:
            shard_size = len(idx_shard)
            # Add the remaining shard to the client with lowest data
            k = min(dict_users, key=lambda x: len(dict_users.get(x)))
            rand_set = set(np.random.choice(idx_shard, shard_size,
                                            replace=False))
            idx_shard = list(set(idx_shard) - rand_set)
            for rand in rand_set:
                ind1 = rand*num_imgs
                ind2 = (rand+1)*num_imgs
                if(len(dict_users[k]) == 0):
                    dict_users[k] = data[ind1: ind2]
                else:
                    dict_users[k] = np.concatenate(
                    (dict_users[k],data[ind1: ind2]),
                    axis=0)
                selected_inds.extend([[ind1, ind2]])
                
                
    return dict_users, selected_inds, data_out



In [22]:
# dict_users = {i: np.array([]) for i in range(4)}

def create_client_iid(image_list, label_list, client_num):    
    max_y = np.argmax(label_list, axis=-1)
    sorted_zip = sorted(zip(max_y, label_list, image_list), key=lambda x: x[0])
    data = [(x,y) for _,y,x in sorted_zip]
    
    dict_users[client_num] = data
    
    return dict_users

In [23]:
# This function differs from all other files sincehere dataframes are being returned instead of list of values
def initializer_income():
    
    x_feats = feature_set1[cols].copy().values
    y_feats = feature_set1[['income']].copy().values

    X_train_new = sc.fit_transform(x_feats)
    y_train_new = y_feats

    
    return [X_train_new, y_train_new]
    
#     x_feats = feature_set1[cols].copy()
#     y_feats = feature_set1[['income']].copy()


#     return [x_feats, y_feats]    

In [43]:
def get_hetero_clients_gender(train_sets):
    
    data = list(zip(train_sets[0], train_sets[1]))
    random.shuffle(data)
    
#     print(type(train_sets), type(train_sets[0]),type(list(list(zip(*data))[0]) ), type(data))
    
    train_sets[0] = list(list(zip(*data))[0])
    train_sets[1] = list(list(zip(*data))[1])
    
    
    # NON_IID
    clients, inds, data_out1 = create_hetero_clients(train_sets[0], train_sets[1], start_client = 0, num_clients=10, initial='client') 
#     print("data_out1")
#     print(type(data_out1))
#     print(data_out1[0])

# #     IID
#     clients = create_client_iid(train_sets[0], train_sets[1], 0)
    
#     clients2 = create_client_iid(train_sets[2], train_sets[3], 1)
#     clients = {**clients, **clients2}
    
#     clients3 = create_client_iid(train_sets[4], train_sets[5], 2)
#     clients = {**clients, **clients3}
    
#     clients4 = create_client_iid(train_sets[6], train_sets[7], 3)
#     clients = {**clients, **clients4}
    

    
    return clients


In [41]:
# take bs = 128 for 5 clients and 10 rounds
def batch_data(data_shard, bs=64):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
#     print(type(data_shard))
    data = []
    label = []
    for x in data_shard:
        data.append(x[0])
        label.append(x[1])
    #seperate shard into data and labels lists
#     data, label = zip(*data_shard)
#     print(type(data[0][0]))
#     print("data: ", data[0])
#     print("label: ", label[0])
    
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
#     print( label[0])
    return dataset.shuffle(len(label)).batch(bs)

In [26]:

#process and batch the training data for each client
def batch_clients(clients):
    clients_batched = dict()
    for (client_name, data) in clients.items():
#         print("data ",type(data))
        clients_batched[client_name] = batch_data(data,126)#non-IID
#         clients_batched[client_name] = batch_data(data,1) #IID
    

    #process and batch the test set  
    test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))
    
#     test_batched
    return clients_batched, test_batched


In [27]:
class SimpleMLP:
    @staticmethod
    def build(shape, classes=2 , learning_rate = 0.001, metric = "accuracy"):

        model = Sequential()
        
        model.add(Dense(128, input_shape = (shape,)))
#         model.add(Dense(128, Activation("relu")))
#         model.add(Dense(64, Activation("relu")))
#         model.add(Dense(32, Activation("relu")))
#         model.add(Dense(1))
        
        model.add(Dense(128, Activation("tanh")))
        model.add(Dense(64, Activation("tanh")))
        model.add(Dense(32, Activation("tanh")))
        model.add(Dense(1,Activation('sigmoid')))
        


        return model

In [28]:
lr = 0.005
comms_round = 10
loss=tf.keras.losses.BinaryCrossentropy(from_logits = False)

metrics = ['binary_accuracy']

optimizer = SGD(learning_rate=lr, 
                decay=lr / comms_round, 
                momentum=0.5
               )     



In [29]:


def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
#     print(len(scaled_weight_list))
    for grad_list_tuple in zip(*scaled_weight_list):
#         print(len(grad_list_tuple))
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    
#     cce = tf.keras.losses.BinaryCrossentropy(from_logits = True)
#     cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    logits = model.predict(X_test)

    score = global_model.evaluate(X_test, y_test, verbose=0)
    acc = score[1] ; loss = score[0]
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))

    return acc, loss

In [30]:
female_features =  feature_set1[feature_set1['gender_Female']==1]
final_female  = pd.DataFrame()
for i in range(10): 
    female_fts = female_features[female_features["fnlwgt"]<= female_quantiles[i]]
    female_fts["fnlwgt"] = quantiles[i]
    
    
    print(female_fts.index)
    female_features = female_features.drop(female_fts.index)
    print("\n")
    print((female_features.shape), final_female.shape)
    final_female = pd.concat([final_female, female_fts])
    print(female_fts["fnlwgt"], female_quantiles[i])
    
male_features = feature_set1[feature_set1['gender_Male']==1] 
final_male  = pd.DataFrame()
for i in range(10): 
    male_fts = male_features[male_features["fnlwgt"]<= male_quantiles[i]]
    male_fts["fnlwgt"] = quantiles[i]
    
    
#     print(male_fts.index)
    male_features = male_features.drop(male_fts.index)
#     print("\n")
#     print((male_features.shape), final_male.shape)
    final_male = pd.concat([final_male, male_fts])
    
    
    repaired_data = pd.DataFrame()
repaired_data = pd.concat([final_male, final_female])
repaired_data = shuffle(repaired_data)
sensitive_atrributes =  [ 'age',
    'race_Amer-Indian-Eskimo',
    'race_Asian-Pac-Islander',
    'race_Black',
    'race_Other',
    'race_White',
    'gender_Female',
    'gender_Male']
for i in sensitive_atrributes: 
    repaired_data[i] = 0
    
    repaired_data = repaired_data.drop('Unnamed: 0', axis = 'columns')

NameError: name 'female_quantiles' is not defined

In [44]:

#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(X_train.shape[1] ,classes=2)
global_model.compile(optimizer=optimizer, loss=loss, metrics=metrics) 


client_names = [0, 1,2,3,4,5,6,7,8,9]


global_loss_list = []
global_freq_list = []
global_acc_list = []
client_loss = {i: np.array([]) for i in range(len(client_names))}
client_accuracy = {i: np.array([]) for i in range(len(client_names))}
client_frequency = {i: np.array([]) for i in range(len(client_names))}

train_sets = initializer_income()
# train_sets = initializer_income_race_gender()

#commence global training loop
for comm_round in range(comms_round):
    epoch_freq = 0
#     clients = create_clients(X_train, y_train, num_clients=15, initial='client')
#     clients = get_hetero_clients()
#     print(type(train_sets[0]))

#     print(train_sets[0]['age'])
    clients = get_hetero_clients_gender(list(train_sets) )
#     clients = get_hetero_clients_gender_race(list(train_sets) )
#     print("client0: ", clients[0])

    clients_batched, test_batched = batch_clients(clients)
#     print("client batched 0: ", clients_batched[1])
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
#     print(client_names)
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(X_train.shape[1],classes=2)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        
        
        
        print("//////////////////////////////")
#         example = list(clients_batched[client].as_numpy_iterator())
#         example[0]['age']
        
        print("clients_batched: ", clients_batched[client])
#                 print("clients: ", (clients[0][0]) )
        
        history = local_model.fit(clients_batched[client], epochs=1, verbose=1)
        
#         get client acc, loss
#         print(client)
        if(len(client_loss[client])== 0):
            client_loss[client] = [history.history['loss'][0]]
            client_accuracy[client] = [history.history['binary_accuracy'][0]]
            client_frequency[client] = [len(clients_batched[client])]
        
        else:
            client_loss[client] = np.append(client_loss[client], (history.history['loss'][0]))
            client_frequency[client] = np.append(client_frequency[client], len(clients_batched[client]))
            client_accuracy[client] = np.append(client_accuracy[client], (history.history['binary_accuracy'][0]))
        
        epoch_freq += len(clients_batched[client])
        
        print("client", client, "loss -->" ,client_loss[client], "freq->", client_frequency[client], "accuracy->", client_accuracy[client])
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)
    
    global_freq_list.append(epoch_freq)
    epoch_freq = 0
    

#     test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)
        global_loss_list.append(global_loss)
        global_acc_list.append(global_acc)
        
        print("global_loss_list: ", global_loss_list )
        print("global_acc_list: ", global_acc_list )
        print("global_freq_list: ", global_freq_list)



<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


data_out1
<class 'list'>
(array([ 0.91646482, -0.04649177, -0.15963972,  1.21496536, -0.03523147,
       -0.14593585, -0.2175757 , -0.03492533,  2.52626483, -0.02771973,
       -0.92136273, -0.11269381, -0.70156267, -0.17891851, -0.179527  ,
       -0.24508517,  2.76555523, -0.01678105, -0.37649823, -0.37618802,
       -0.17815534, -0.21117047, -0.2554779 , -0.33653279, -0.07082154,
       -0.38135638, -0.14300071, -0.35746063, -0.17351455, -0.22532901,
       -0.82412796,  1.69528279, -0.17654328, -0.42836007, -0.34117932,
       -0.22419805, -0.24449789,  5.78410825, -0.26401111, -0.0159999 ,
       -1.50719061, -0.18939262, -0.29443044, -0.20406156, -0.02086318,
       -0.09870466, -0.17792583, -0.32481669, -0.09115626,  0.41017233,
       -0.70282207,  0.70282207, -0.13427492, -0.02426908, -0.0595346 ,
       -0.04962853, -0.03986597, -0.05456777, -0.04501058, -0.02994265,
       -0.05732963, -0.04831574, -0.02771973, -0.0639213 , -0.031609  ,
       -0.04296634, -0.03888801,  0.  

KeyboardInterrupt: 

In [31]:
feature_set1

,Unnamed: 0,age,fnlwgt,education,educational-num,capital-gain,capital-loss,hours-per-week,income,marital-status_Divorced,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,38914,25,178960,1.0,7,0,0,40,0,0,...,0,0,0,0,0,0,0,1,0,0
1,5303,31,158672,1.0,7,0,0,38,0,0,...,0,1,0,0,0,0,0,0,0,0
2,37481,17,25051,0.0,6,0,0,16,0,0,...,0,0,0,0,0,0,0,1,0,0
3,33175,50,231196,9.0,13,0,0,50,0,1,...,0,0,0,0,0,0,0,1,0,0
4,30045,22,187052,11.0,9,0,0,40,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39068,43280,60,190682,8.0,11,0,0,37,0,0,...,0,0,0,0,0,0,0,1,0,0
39069,18495,64,177825,11.0,9,1055,0,40,0,0,...,0,0,0,0,0,0,0,1,0,0
39070,27726,49,192323,11.0,9,0,0,40,0,0,...,0,0,0,0,0,0,0,1,0,0
39071,25866,17,156736,0.0,6,0,0,12,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats 

pdfs = []
for i in list(client_accuracy.keys()):
    print("client_",i, " loss: ", client_loss[i],"freq: ", client_frequency[i], "acc", client_accuracy[i])
    
#     norm
#     mean, std = stats.norm.fit(client_loss[i])
#     pdf =  stats.norm.pdf(client_loss[i], mean, std)

#     df_mean = np.mean(client_frequency[i])
#     df_std = np.std(client_frequency[i])
#     pdf = stats.norm.pdf(client_frequency[i], df_mean, df_std)
    
#     lognorm
#     shape, loc, scale = stats.lognorm.fit(client_loss[i])
#     pdf = stats.lognorm.pdf(client_loss[i], shape, loc, scale)

#     beta
#     beta_params = stats.beta.fit(client_loss[i])
#     pdf = stats.beta.pdf(client_loss[i], beta_params[0], beta_params[1], beta_params[2], beta_params[3])

#     burr
#     burr_params = stats.burr.fit(client_loss[i])
#     pdf = stats.burr.pdf(client_loss[i], burr_params[0], burr_params[1], burr_params[2], burr_params[3])

    
#  gamma
    shape, loc, scale = stats.gamma.fit(client_loss[i])
    pdf = stats.gamma.cdf(client_loss[i], shape, loc=loc, scale=scale)
    
    pdfs.append(pdf)
    sns.set_style("whitegrid")

    ax = sns.displot( x = pdf, kind = "kde", height=10, aspect=2,
                linewidth = 5 )
    plt.ylabel("LOSS")
#     plt.xlabel("Loss")
    plt.xlabel("CDF")
    
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats 

for i in list(client_accuracy.keys()):
    print("client_",i, "\nloss: ", client_loss[i],"\nfreq: ", client_frequency[i], "\nacc", client_accuracy[i])

In [ ]:
print("_GAMMA_")

cdfs_loss_gamma = []
cdfs_acc_gamma = []
cdfs_freq_gamma = []
for i in list(client_accuracy.keys()):
#     print("client_",i, "\nloss: ", client_loss[i],"\nfreq: ", client_frequency[i], "\nacc", client_accuracy[i])

    shape, loc, scale = stats.gamma.fit(client_loss[i])
    cdf_loss_gamma =  stats.gamma.cdf(client_loss[i], shape, loc, scale)
    cdfs_loss_gamma.append(cdf_loss_gamma)

    shape, loc, scale= stats.gamma.fit(client_accuracy[i])
    cdf_acc_gamma = stats.gamma.cdf(client_accuracy[i],shape, loc, scale)
    cdfs_acc_gamma.append(cdf_acc_gamma)

    shape, loc, scale  = stats.gamma.fit(client_frequency[i])
    cdf_freq_gamma = stats.gamma.cdf(client_frequency[i],shape, loc, scale )
    cdfs_freq_gamma.append(cdf_freq_gamma)


print("ACCURACY")
for i in range(len(cdfs_acc_gamma)):
    for j in range(i+1,len(cdfs_acc_gamma)):
        print(i," and ", j , stats.wasserstein_distance(cdfs_acc_gamma[i], cdfs_acc_gamma[j]))


shape, loc, scale = stats.gamma.fit(global_acc_list)
cdf_global_acc_gamma =  stats.gamma.cdf(global_acc_list, shape, loc, scale)

for i in range(len(cdfs_acc_gamma)):
    print(i," and global: " , stats.wasserstein_distance(cdfs_acc_gamma[i], cdf_global_acc_gamma))

print("LOSS")
for i in range(len(cdfs_loss_gamma)):
    for j in range(i+1,len(cdfs_loss_gamma)):
        print(i," and ", j , stats.wasserstein_distance(cdfs_loss_gamma[i], cdfs_loss_gamma[j]))


shape, loc, scale = stats.gamma.fit(global_loss_list)
cdf_global_loss_gamma =  stats.gamma.cdf(global_loss_list, shape, loc, scale)

for i in range(len(cdfs_loss_gamma)):
    print(i," and global: " , stats.wasserstein_distance(cdfs_loss_gamma[i], cdf_global_loss_gamma))

print("FREQUENCY")
for i in range(len(cdfs_freq_gamma)):
    for j in range(i+1,len(cdfs_freq_gamma)):
        print(i," and ", j , stats.wasserstein_distance(cdfs_freq_gamma[i], cdfs_freq_gamma[j]))


shape, loc, scale = stats.gamma.fit(global_freq_list)
cdf_global_freq_gamma =  stats.gamma.cdf(global_freq_list,shape, loc, scale)

for i in range(len(cdfs_freq_gamma)):
    print(i," and global: " , stats.wasserstein_distance(cdfs_freq_gamma[i], cdf_global_freq_gamma))

In [ ]:

# for i in range(len(pdfs_loss_gamma)):
#     print(i," and global: " , stats.wasserstein_distance(pdfs_loss_gamma[i], pdf_global_loss_gamma))
#     M = ot.dist(pdfs_loss_gamma[i], pdf_global_loss_gamma, metric='euclidean')
#     W = ot.emd2(pdfs_loss_gamma[i], pdf_global_loss_gamma, M)
#     print("actual ", W)

In [ ]:
print("_NORM_")

pdfs_loss_norm = []
pdfs_acc_norm = []
pdfs_freq_norm = []
for i in list(client_accuracy.keys()):
#     print("client_",i, " loss: ", client_loss[i],"freq: ", client_frequency[i], "acc", client_accuracy[i])
    
    mean, std = stats.norm.fit(client_loss[i])
    pdf_loss_norm =  stats.norm.pdf(client_loss[i], mean, std)
    pdfs_loss_norm.append(pdf_loss_norm)

    mean, std = stats.norm.fit(client_accuracy[i])
    pdf_acc_norm = stats.norm.pdf(client_accuracy[i],mean, std )
    pdfs_acc_norm.append(pdf_acc_norm)
    
    mean, std  = stats.norm.fit(client_frequency[i])
    pdf_freq_norm = stats.norm.pdf(client_frequency[i], mean, std )
    pdfs_freq_norm.append(pdf_freq_norm)
    
    
print("ACCURACY")
for i in range(len(pdfs_acc_norm)):
    for j in range(i+1,len(pdfs_acc_norm)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_acc_norm[i], pdfs_acc_norm[j]))
        
        
mean, std = stats.norm.fit(global_acc_list)
pdf_global_acc_norm =  stats.norm.pdf(global_acc_list, mean, std)

for i in range(len(pdfs_acc_norm)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_acc_norm[i], pdf_global_acc_norm))
    
print("LOSS")
for i in range(len(pdfs_loss_norm)):
    for j in range(i+1,len(pdfs_loss_norm)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_loss_norm[i], pdfs_loss_norm[j]))
        
        
mean, std = stats.norm.fit(global_loss_list)
pdf_global_loss_norm =  stats.norm.pdf(global_loss_list, mean, std)

for i in range(len(pdfs_loss_norm)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_loss_norm[i], pdf_global_loss_norm))
    
print("FREQUENCY")
for i in range(len(pdfs_freq_norm)):
    for j in range(i+1,len(pdfs_freq_norm)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_freq_norm[i], pdfs_freq_norm[j]))
        
        
mean, std = stats.norm.fit(global_freq_list)
pdf_global_freq_norm =  stats.norm.pdf(global_freq_list, mean, std)

for i in range(len(pdfs_freq_norm)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_freq_norm[i], pdf_global_freq_norm))


In [ ]:
print("_LOGNORM_")

pdfs_loss = []
pdfs_acc = []
pdfs_freq = []
for i in list(client_accuracy.keys()):
#     print("client_",i, "\nloss: ", client_loss[i],"\nfreq: ", client_frequency[i], "\nacc", client_accuracy[i])

    shape, loc, scale = stats.lognorm.fit(client_loss[i])
    pdf_loss =  stats.lognorm.pdf(client_loss[i], shape, loc, scale)
    pdfs_loss.append(pdf_loss)

    shape, loc, scale= stats.lognorm.fit(client_accuracy[i])
    pdf_acc = stats.lognorm.pdf(client_accuracy[i],shape, loc, scale)
    pdfs_acc.append(pdf_acc)

    shape, loc, scale  = stats.lognorm.fit(client_frequency[i])
    pdf_freq = stats.lognorm.pdf(client_frequency[i],shape, loc, scale )
    pdfs_freq.append(pdf_freq)


print("ACCURACY")
for i in range(len(pdfs_acc)):
    for j in range(i+1,len(pdfs_acc)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_acc[i], pdfs_acc[j]))


shape, loc, scale = stats.lognorm.fit(global_acc_list)
pdf_global_acc =  stats.lognorm.pdf(global_acc_list, shape, loc, scale)

for i in range(len(pdfs_acc)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_acc[i], pdf_global_acc))

print("LOSS")
for i in range(len(pdfs_loss)):
    for j in range(i+1,len(pdfs_loss)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_loss[i], pdfs_loss[j]))


shape, loc, scale = stats.lognorm.fit(global_loss_list)
pdf_global_loss =  stats.lognorm.pdf(global_loss_list, shape, loc, scale)

for i in range(len(pdfs_loss)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_loss[i], pdf_global_loss))

print("FREQUENCY")
for i in range(len(pdfs_freq)):
    for j in range(i+1,len(pdfs_freq)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_freq[i], pdfs_freq[j]))


shape, loc, scale = stats.lognorm.fit(global_freq_list)
pdf_global_freq =  stats.lognorm.pdf(global_freq_list,shape, loc, scale)

for i in range(len(pdfs_freq)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_freq[i], pdf_global_freq))

In [ ]:
print("_BETA_")

pdfs_loss = []
pdfs_acc = []
pdfs_freq = []
for i in list(client_accuracy.keys()):

    beta_params = stats.beta.fit(client_loss[i])
    pdf_loss =  stats.beta.pdf(client_loss[i],beta_params[0], beta_params[1], beta_params[2], beta_params[3])
    pdfs_loss.append(pdf_loss)

    beta_params = stats.beta.fit(client_accuracy[i])
    pdf_acc = stats.beta.pdf(client_accuracy[i],beta_params[0], beta_params[1], beta_params[2], beta_params[3])
    pdfs_acc.append(pdf_acc)

    beta_params  = stats.beta.fit(client_frequency[i])
    pdf_freq = stats.beta.pdf(client_frequency[i],beta_params[0], beta_params[1], beta_params[2], beta_params[3])
    pdfs_freq.append(pdf_freq)


print("ACCURACY")
for i in range(len(pdfs_acc)):
    for j in range(i+1,len(pdfs_acc)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_acc[i], pdfs_acc[j]))


beta_params = stats.beta.fit(global_acc_list)
pdf_global_acc =  stats.beta.pdf(global_acc_list, beta_params[0], beta_params[1], beta_params[2], beta_params[3])

for i in range(len(pdfs_acc)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_acc[i], pdf_global_acc))

print("LOSS")
for i in range(len(pdfs_loss)):
    for j in range(i+1,len(pdfs_loss)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_loss[i], pdfs_loss[j]))


beta_params = stats.beta.fit(global_loss_list)
pdf_global_loss =  stats.beta.pdf(global_loss_list, beta_params[0], beta_params[1], beta_params[2], beta_params[3])

for i in range(len(pdfs_loss)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_loss[i], pdf_global_loss))

print("FREQUENCY")
for i in range(len(pdfs_freq)):
    for j in range(i+1,len(pdfs_freq)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_freq[i], pdfs_freq[j]))


beta_params = stats.beta.fit(global_freq_list)
pdf_global_freq =  stats.beta.pdf(global_freq_list,beta_params[0], beta_params[1], beta_params[2], beta_params[3])

for i in range(len(pdfs_freq)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_freq[i], pdf_global_freq))

In [ ]:
print("_BURR_")

pdfs_loss = []
pdfs_acc = []
pdfs_freq = []
for i in list(client_accuracy.keys()):

    params = stats.burr.fit(client_loss[i])
    pdf_loss =  stats.burr.pdf(client_loss[i],params[0], params[1], params[2], params[3])
    pdfs_loss.append(pdf_loss)

    beta_params = stats.burr.fit(client_accuracy[i])
    pdf_acc = stats.burr.pdf(client_accuracy[i],beta_params[0], beta_params[1], beta_params[2], beta_params[3])
    pdfs_acc.append(pdf_acc)

    beta_params  = stats.burr.fit(client_frequency[i])
    pdf_freq = stats.burr.pdf(client_frequency[i],beta_params[0], beta_params[1], beta_params[2], beta_params[3])
    pdfs_freq.append(pdf_freq)


print("ACCURACY")
for i in range(len(pdfs_acc)):
    for j in range(i+1,len(pdfs_acc)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_acc[i], pdfs_acc[j]))


beta_params = stats.burr.fit(global_acc_list)
pdf_global_acc =  stats.burr.pdf(global_acc_list, beta_params[0], beta_params[1], beta_params[2], beta_params[3])

for i in range(len(pdfs_acc)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_acc[i], pdf_global_acc))

print("LOSS")
for i in range(len(pdfs_loss)):
    for j in range(i+1,len(pdfs_loss)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_loss[i], pdfs_loss[j]))


beta_params = stats.burr.fit(global_loss_list)
pdf_global_loss =  stats.burr.pdf(global_loss_list, beta_params[0], beta_params[1], beta_params[2], beta_params[3])

for i in range(len(pdfs_loss)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_loss[i], pdf_global_loss))

print("FREQUENCY")
for i in range(len(pdfs_freq)):
    for j in range(i+1,len(pdfs_freq)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_freq[i], pdfs_freq[j]))


beta_params = stats.burr.fit(global_freq_list)
pdf_global_freq =  stats.burr.pdf(global_freq_list,beta_params[0], beta_params[1], beta_params[2], beta_params[3])

for i in range(len(pdfs_freq)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_freq[i], pdf_global_freq))

In [ ]:
for i in list(client_loss.keys()):
#     if(i== len(list(client_loss.keys()))-1):
#         continue
    print("client_",i, "loss: ", client_loss[i])

In [ ]:
for i in list(client_loss.keys()):
#     if(i== len(list(client_loss.keys()))-1):
#         continue
    print("client_",i, "freq: ", client_frequency[i])

In [ ]:
for i in range(len(pdfs)):
    for j in range(i+1,len(pdfs)):
        print(i," and ", j , stats.wasserstein_distance(pdfs[i], pdfs[j]))

In [ ]:
# # gamma
shape, loc, scale = stats.gamma.fit(global_freq_list)
pdf_global = stats.gamma.pdf(global_freq_list, shape, loc=loc, scale=scale)

# df_mean = np.mean(global_loss_list)
# df_std = np.std(global_loss_list)
# pdf_global = stats.norm.pdf(global_loss_list, df_mean, df_std)

# mean, std = stats.norm.fit(global_freq_list)
# pdf_global =  stats.norm.pdf(global_freq_list, mean, std)

# lognorm
# shape, loc, scale = stats.lognorm.fit(global_loss_list)
# pdf_global = stats.lognorm.pdf(global_loss_list, shape, loc, scale)

# global_acc_list
# global_freq_list

# burr
# burr_params = stats.beta.fit(global_freq_list)
# pdf_global = stats.beta.pdf(global_freq_list, burr_params[0], burr_params[1], burr_params[2], burr_params[3])

for i in range(len(pdfs)):
    print(i," and global: " , stats.wasserstein_distance(pdfs[i], pdf_global))

In [ ]:
!pip install fitter

In [ ]:
# from fitter import Fitter, get_common_distributions, get_distributions

# for i in list(client_loss.keys()):
#     height = client_loss[i]

#     f = Fitter(height,
#                distributions=['gamma',
#                               'lognorm',
#                               "beta",
#                               "burr",
#                               "norm"])
#     f.fit()
#     f.summary()

#     sns.set_style('white')
#     sns.set_context("paper", font_scale = 2)
#     sns.displot(data=dataset, x="Height", kind="hist", bins = 100, aspect = 1.5)

In [ ]:
# 8 clients---> gender, race, income
pdfs = []
for i in list(client_loss.keys()):
    if(i== len(list(client_loss.keys()))-1):
        continue
    print("client_",i, " loss: ", client_loss[i],"freq: ", client_frequency[i])

#     shape, loc, scale = stats.gamma.fit(client_loss[i])
    
#     # Printing the estimated parameters
#     print("Shape:", shape)
#     print("Location:", loc)
#     print("Scale:", scale)
#     pdf = stats.gamma.pdf(client_loss[i], shape, loc=loc, scale=scale)
    
    
    df_mean = np.mean(client_loss[i])
    df_std = np.std(client_loss[i])
    pdf = stats.norm.pdf(client_loss[i], df_mean, df_std)
    pdfs.append(pdf)
    sns.set_style("whitegrid")

    ax = sns.displot( x = pdf, kind = "kde", height=10, aspect=2,
                linewidth = 5 )
    ax.fig.suptitle('Original distribution', size = 20)
    plt.ylabel("Frequency")
    plt.xlabel("Loss")
    plt.show()

In [ ]:
for i in range(len(pdfs)):
    for j in range(i+1,len(pdfs)):
        print(i," and ", j , stats.wasserstein_distance(pdfs[i], pdfs[j]))

In [ ]:
shape, loc, scale = stats.gamma.fit(global_loss_list)
pdf_global = stats.gamma.pdf(global_loss_list, shape, loc=loc, scale=scale)

for i in range(len(pdfs)):
    print(i," and global: " , stats.wasserstein_distance(pdfs[i], pdf_global))

In [ ]:
# experiment plotter

# for i in list(client_loss.keys()):
#     if(i== len(list(client_loss.keys()))-1):
#         continue
#     print("client_",i, " loss: ", client_loss[i],"freq: ", client_frequency[i])
    
# #     df_mean = np.mean(client_loss[i])
# #     df_std = np.std(client_loss[i])
# #     pdf = stats.norm.pdf(client_loss[i], df_mean, df_std)

#     shape, loc, scale = stats.gamma.fit(client_loss[i])
    
#     # Printing the estimated parameters
# #     print("Shape:", shape)
# #     print("Location:", loc)
# #     print("Scale:", scale)
#     pdf = stats.gamma.pdf(client_loss[i], shape, loc=loc, scale=scale)

# #     plt.plot(client_loss[i], pdf, "-o", label = i)

# #     client_loss[i] = stats.gamma.rvs(1, size=5000)+5
#     sns.set_style("whitegrid")

#     ax = sns.displot( x = pdf, kind = "kde", height=10, aspect=2,
#                 linewidth = 5 )
#     ax.fig.suptitle('Original distribution', size = 20)
# #     plt.plot(client_loss[i], client_frequency[i], "-o", label = i)
# #     plt.legend()
#     plt.ylabel("Frequency")
#     plt.xlabel("Loss")
#     plt.show()

In [ ]:
score = global_model.evaluate(X_test, y_test, verbose=0)
score

In [ ]:
nn_preds = global_model.predict(X_test)
nn_preds = (nn_preds > 0.5)

nn_precision =precision_score(y_test, nn_preds)
nn_recall = recall_score(y_test, nn_preds)
nn_accuracy = accuracy_score(y_test, nn_preds)
nn_f1 = f1_score(y_test, nn_preds)


print("Precision = {}".format(nn_precision))
print("Recall = {}".format(nn_recall))
print("Accuracy = {}".format(nn_accuracy))
print("f1 = {}".format(nn_f1))


In [ ]:
arr = nn_preds > 0.5

unique, counts = np.unique(arr, return_counts=True)

np.asarray((unique, counts)).T

In [ ]:
# pdf for client losses
# x-loss
# y- frequency
# each client has one pdf for all rounds
# using histogram


# drop client
# non iid
# fedavg


# gender dist
# kernel density


# non-iid, loss to accuracy
# 4client